In [1]:
import os
import pandas as pd
from Data_loader import Dataset_Custom
import argparse
import warnings
from tools import EarlyStopping
from torch.utils.data import Dataset, DataLoader
from utils import get_data
from Model import ANN
from torch import nn
import torch
import copy
from tqdm import tqdm
import random 
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

seed_everything(0)

In [3]:
from Server import  Server
from Clients import Client
from Train import Trainer

In [4]:
parser_train = argparse.ArgumentParser(description='FL')
parser_train.add_argument('--root_path', type=str, default='../Data/GFC12/')
parser_train.add_argument('--dataset_paths', type=list, default=["wf1", "wf2", "wf3", "wf4", "wf5", "wf6", "wf7"])
parser_train.add_argument('--number_clients', type=int, default=7)
parser_train.add_argument('--seq_len', type=int, default=24*4)
parser_train.add_argument('--pred_len', type=int, default=4)
parser_train.add_argument('--label_len', type=int, default=0)
parser_train.add_argument('--train_length', type=int, default=16800)
parser_train.add_argument('--target', type=str, default='target')
parser_train.add_argument('--scale', type=bool, default=True)
parser_train.add_argument('--inverse', type=bool, default=True)

parser_train.add_argument('--lr', type=float, default=1e-4)
parser_train.add_argument('--global_epochs', type=int, default=200)
parser_train.add_argument('--local_epochs', type=int, default=1)
parser_train.add_argument('--fine_tune_epochs', type=int, default=20)
parser_train.add_argument('--patience', type=int, default=3)
parser_train.add_argument('--fed_patience', type=int, default=3)
parser_train.add_argument('--hidden_layers', type=list, default=[64,64,64])
parser_train.add_argument('--input_size', type=int, default=293)
parser_train.add_argument('--output_size', type=int, default=9)

parser_train.add_argument('--fine_tune_lr', type=float, default=1e-5)
parser_train.add_argument('--ensemble_flag', type=bool, default=True)
parser_train.add_argument('--batch_size', type=int, default=64)
parser_train.add_argument('--device', type=str, default='cuda:2' if torch.cuda.is_available() else 'cpu')
parser_train.add_argument('--forecasting_mode', type=str, default='prob')
parser_train.add_argument('--model_type', type=str, default='NN')
parser_train.add_argument('--model_save_path', type=str, default='../Model12/', help='location of model checkpoints')
parser_train.add_argument('--quantiles', type=list, default=[0.1,0.2,0.3,0.4, 0.5,0.6,0.7,0.8, 0.9])
args_train = parser_train.parse_args(args=[])


In [5]:
seed_everything(0)

In [6]:
clients=[]
for path in tqdm(args_train.dataset_paths):
    args_temp=copy.deepcopy(args_train)
    args_temp.dataset_paths=path
    clients.append(Client(args_temp))

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:05<00:00,  1.34it/s]


In [7]:
server = Server(args_train,clients)

In [8]:
server.fed_train()

test performance: [0.16786858957413942, 0.1676235998891396, 0.19591147434731868, 0.18001745878527425, 0.172340816142012, 0.18452763549779375, 0.18163522973350466]
Epoch: 0 | Loss: 0.0851
Epoch: 0 | Loss: 0.0677
Epoch: 0 | Loss: 0.1082
Epoch: 0 | Loss: 0.0842
Epoch: 0 | Loss: 0.0824
Epoch: 0 | Loss: 0.0744
Epoch: 0 | Loss: 0.0874
Federated training Epoch [1/200] Val Loss: 0.0725
test performance: [0.07624770001801726, 0.08147190712800581, 0.09445986327753492, 0.08585161174813362, 0.08840716811381791, 0.08492420858716311, 0.08964639950594673]
Epoch: 0 | Loss: 0.0632
Epoch: 0 | Loss: 0.0702
Epoch: 0 | Loss: 0.0851
Epoch: 0 | Loss: 0.0685
Epoch: 0 | Loss: 0.0681
Epoch: 0 | Loss: 0.0672
Epoch: 0 | Loss: 0.0832
Federated training Epoch [2/200] Val Loss: 0.0634
test performance: [0.06760701211807255, 0.07407047186199933, 0.08220775467534996, 0.07557713283761723, 0.0785346180838469, 0.07411682624880174, 0.07904363443998441]
Epoch: 0 | Loss: 0.0529
Epoch: 0 | Loss: 0.0669
Epoch: 0 | Loss: 0.056

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [9]:
server.central_train()

Epoch [1/200] Train Loss: 0.0944 Val Loss: 0.0712
Epoch [2/200] Train Loss: 0.0673 Val Loss: 0.0589
Epoch [3/200] Train Loss: 0.0578 Val Loss: 0.0543
Epoch [4/200] Train Loss: 0.0538 Val Loss: 0.0515
Epoch [5/200] Train Loss: 0.0515 Val Loss: 0.0496
Epoch [6/200] Train Loss: 0.0504 Val Loss: 0.0498
Epoch [7/200] Train Loss: 0.0494 Val Loss: 0.0478
Epoch [8/200] Train Loss: 0.0488 Val Loss: 0.0477
Epoch [9/200] Train Loss: 0.0486 Val Loss: 0.0479
Epoch [10/200] Train Loss: 0.0479 Val Loss: 0.0468
Epoch [11/200] Train Loss: 0.0479 Val Loss: 0.0465
Epoch [12/200] Train Loss: 0.0473 Val Loss: 0.0462
Epoch [13/200] Train Loss: 0.0472 Val Loss: 0.0461
Epoch [14/200] Train Loss: 0.0468 Val Loss: 0.0463
Epoch [15/200] Train Loss: 0.0465 Val Loss: 0.0455
Epoch [16/200] Train Loss: 0.0464 Val Loss: 0.0455
Epoch [17/200] Train Loss: 0.0464 Val Loss: 0.0452
Epoch [18/200] Train Loss: 0.0462 Val Loss: 0.0451
Epoch [19/200] Train Loss: 0.0460 Val Loss: 0.0450
Epoch [20/200] Train Loss: 0.0459 Val Lo

ANN(
  (hidden_layers): ModuleList(
    (0): Linear(in_features=293, out_features=64, bias=True)
    (1-2): 2 x Linear(in_features=64, out_features=64, bias=True)
  )
  (relu): ReLU()
  (output): Linear(in_features=64, out_features=9, bias=True)
)

In [10]:
server.local_train()

Launch Local Training!
Epoch [1/200] Train Loss: 0.0863 Val Loss: 0.0703
Epoch [2/200] Train Loss: 0.0664 Val Loss: 0.0589
Epoch [3/200] Train Loss: 0.0571 Val Loss: 0.0538
Epoch [4/200] Train Loss: 0.0534 Val Loss: 0.0509
Epoch [5/200] Train Loss: 0.0513 Val Loss: 0.0494
Epoch [6/200] Train Loss: 0.0501 Val Loss: 0.0484
Epoch [7/200] Train Loss: 0.0494 Val Loss: 0.0479
Epoch [8/200] Train Loss: 0.0488 Val Loss: 0.0474
Epoch [9/200] Train Loss: 0.0482 Val Loss: 0.0469
Epoch [10/200] Train Loss: 0.0479 Val Loss: 0.0467
Epoch [11/200] Train Loss: 0.0477 Val Loss: 0.0464
Epoch [12/200] Train Loss: 0.0473 Val Loss: 0.0462
Epoch [13/200] Train Loss: 0.0470 Val Loss: 0.0467
Epoch [14/200] Train Loss: 0.0467 Val Loss: 0.0458
Epoch [15/200] Train Loss: 0.0465 Val Loss: 0.0459
Epoch [16/200] Train Loss: 0.0463 Val Loss: 0.0454
Epoch [17/200] Train Loss: 0.0462 Val Loss: 0.0453
Epoch [18/200] Train Loss: 0.0460 Val Loss: 0.0451
Epoch [19/200] Train Loss: 0.0458 Val Loss: 0.0451
Epoch [20/200] Tr

In [11]:
local_fine_tune_losses=[]
local_fine_tune_preds=[]
local_fine_tune_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    local_fine_tune_pred,local_fine_tune_loss,local_fine_tune_model=clients[i].local_fine_tune()
    local_fine_tune_losses.append(local_fine_tune_loss)
    local_fine_tune_preds.append(local_fine_tune_pred)
    local_fine_tune_models.append(local_fine_tune_model)
print(local_fine_tune_losses)

1e-05 0
Epoch [1/20] Train Loss: 0.0451 Val Loss: 0.0445
Epoch [2/20] Train Loss: 0.0451 Val Loss: 0.0442
Epoch [3/20] Train Loss: 0.0451 Val Loss: 0.0442
Epoch [4/20] Train Loss: 0.0450 Val Loss: 0.0442
Epoch [5/20] Train Loss: 0.0450 Val Loss: 0.0442
Epoch [6/20] Train Loss: 0.0449 Val Loss: 0.0442
Epoch [7/20] Train Loss: 0.0449 Val Loss: 0.0441
Epoch [8/20] Train Loss: 0.0449 Val Loss: 0.0442
Epoch [9/20] Train Loss: 0.0448 Val Loss: 0.0441
Epoch [10/20] Train Loss: 0.0448 Val Loss: 0.0445
Epoch [11/20] Train Loss: 0.0448 Val Loss: 0.0441
Epoch [12/20] Train Loss: 0.0447 Val Loss: 0.0440
Epoch [13/20] Train Loss: 0.0447 Val Loss: 0.0440
Epoch [14/20] Train Loss: 0.0447 Val Loss: 0.0440
Epoch [15/20] Train Loss: 0.0447 Val Loss: 0.0442
Epoch [16/20] Train Loss: 0.0446 Val Loss: 0.0440
Epoch [17/20] Train Loss: 0.0446 Val Loss: 0.0439
Epoch [18/20] Train Loss: 0.0446 Val Loss: 0.0439
Epoch [19/20] Train Loss: 0.0446 Val Loss: 0.0439
Epoch [20/20] Train Loss: 0.0446 Val Loss: 0.0439
1

In [12]:
fed_local_losses=[]
fed_local_preds=[]
fed_local_models=[]
seed_everything(0)
for i in range(args_train.number_clients):
    fed_local_pred,fed_local_loss,fed_local_model=clients[i].fed_local_evaluation()
    fed_local_losses.append(fed_local_loss)
    fed_local_preds.append(fed_local_pred)
    fed_local_models.append(fed_local_model)
print(fed_local_losses)
print(np.mean(fed_local_losses))

[0.04738055886573171, 0.05263746971238966, 0.05798239472692143, 0.05267014441220728, 0.05327578918523576, 0.05104644261683299, 0.05414957301818753]
0.05273462464821519


In [13]:
local_losses=[]
local_preds=[]
local_models=[]
for i in range(args_train.number_clients):
    local_pred,local_loss,local_model=clients[i].local_evaluation()
    local_losses.append(local_loss)
    local_preds.append(local_pred)
    local_models.append(local_model)
print(local_losses)

[0.047523875287032294, 0.053075658706055115, 0.0580338093337335, 0.05174410740576991, 0.05369003568712163, 0.05185019247846244, 0.05432962053391623]


In [14]:
central_losses=[]
central_preds=[]
central_models=[]
for i in range(args_train.number_clients):
    central_pred,central_loss,central_model=server.central_evaluation(dataset=i)
    central_losses.append(central_loss)
    central_preds.append(central_pred)
    central_models.append(central_model)
print(central_losses)

[0.04773888712085477, 0.055938527995899115, 0.05991943744457748, 0.053290099919811915, 0.055807694607758765, 0.05220189785635839, 0.056752502771528206]


In [17]:
print(local_losses)
print(central_losses)
print(fed_local_losses)
print(local_fine_tune_losses)

[0.047523875287032294, 0.053075658706055115, 0.0580338093337335, 0.05174410740576991, 0.05369003568712163, 0.05185019247846244, 0.05432962053391623]
[0.04773888712085477, 0.055938527995899115, 0.05991943744457748, 0.053290099919811915, 0.055807694607758765, 0.05220189785635839, 0.056752502771528206]
[0.04738055886573171, 0.05263746971238966, 0.05798239472692143, 0.05267014441220728, 0.05327578918523576, 0.05104644261683299, 0.05414957301818753]
[0.04672484316748299, 0.05219567769959773, 0.05765293501572658, 0.05168611156971079, 0.05303531432562597, 0.05059497972491057, 0.05391620361080317]


In [16]:
df = pd.DataFrame({
    'local_losses': local_losses,
    'central_losses': central_losses,
    'fed_local_losses': fed_local_losses,
    'local_fine_tune_losses': local_fine_tune_losses
})
df.T.to_csv('losses.csv', index=False)

In [18]:
import pickle

# Save the server object
with open('../result/4/server_benchmark.pkl', 'wb') as f:
    pickle.dump(server, f)

# Save the clients object
with open('../result/4/clients_benchmark.pkl', 'wb') as f:
    pickle.dump(clients, f)

In [334]:
import pickle

# Save server and clients
with open('server.pkl', 'wb') as f:
    pickle.dump(server, f)

with open('clients.pkl', 'wb') as f:
    pickle.dump(clients, f)

# Load server and clients
with open('server.pkl', 'rb') as f:
    server = pickle.load(f)

with open('clients.pkl', 'rb') as f:
    clients = pickle.load(f)